# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

## Inicialización de librerías

In [27]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import re
import math


## Petición al servidor

In [ ]:


url="https://www.aemet.es/es/eltiempo/observacion/ultimosdatos?k=and&w=0&datos=det&x=h24&f=temperatura"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
response.status_code

## Extracción y limpieza de datos

In [ ]:
table = soup.find('table')
headers_row = table.find('tr')
headers = []
if headers_row:
    for th in headers_row.find_all(['th', 'td']):
        headers.append(th.get_text(strip=True))
data_rows = []
rows = table.find_all('tr')[1:]      
for row in rows:
        cells = row.find_all(['td', 'th'])
        row_data = []
        for cell in cells:
            text = cell.get_text(strip=True)
            
            duplicate_pattern = r'(\d+\.\d+)\1+'
            text = re.sub(duplicate_pattern, r'\1', text)
        
        #Manejar enteros duplicados
            duplicate_int_pattern = r'(\d{1,2})\1+'
            text = re.sub(duplicate_int_pattern, r'\1', text)
        
        #Buscar números en el texto limpio
            number_match = re.search(r'(-?\d+[.]?\d*)', text)
        
            if number_match:
                number_str = number_match.group(1)
            
            try:
                #Si tiene punto decimal
                if '.' in number_str:
                    number = float(number_str)
                    formatted_text = f"{number:.1f}"
                else:
                    #Es entero
                    number = int(number_str)
                    #Limitar enteros a máximo 99
                    if number > 99:
                        formatted_text = "99"
                    elif number < -99:
                        formatted_text = "-99"
                    else:
                        formatted_text = str(number)
            except ValueError:
                formatted_text = text
            else:
                formatted_text = text  #Mantener texto original si no es número
            
            row_data.append(formatted_text)
            
        if row_data: 
            data_rows.append(row_data)


df = pd.DataFrame(data_rows, columns=headers)
df.set_index('Estación', inplace=True)
df

,Provincia,Temp. (°C),V. vien. (km/h),Dir. viento,Racha (km/h),Dir. racha,Prec. (mm),Presión (hPa),Tend. (hPa),Humedad (%)
Estación,,,,,,,,,,
Abla,Almería,20.2,0,1-Calma,5,9-Noroeste,0.0,916.0,-0.4-0.4,38.0
Adra,Almería,21.3,17,8-Oeste,25,8-Oeste,0.0,1013.3,-0.2-0.2,86.0
Aguilar de la Frontera,Córdoba,17.7,17,8-Oeste,23,8-Oeste,0.0,,,81.0
Alajar,Huelva,16.6,13,2-Norte,24,3-Nordeste,0.0,950.3,1.0,62.0
Albox,Almería,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
Villanueva de Córdoba,Córdoba,20.3,13,6-Sur,28,7-Sudoeste,0.0,929.8,0.3,64.0
Villanueva del Arzobispo,Jaén,,,,,,,,,
Villarrasa,Huelva,19.1,12,2-Norte,22,2-Norte,0.0,,,58.0


## Conversión de los nombres de las columnas

In [39]:
df = df.rename(columns={
    'Provincia': 'provincia',
    'Temp. (°C)': 'temperatura',
    'V. vien. (km/h)': 'velocidad_viento',
    'Dir. viento': 'direccion_viento',
    'Racha (km/h)': 'racha_maxima',
    'Dir. racha': 'direccion_racha',
    'Prec. (mm)': 'precipitacion',
    'Presión (hPa)': 'presion',
    'Tend. (hPa)': 'tendencia_presion',
    'Humedad (%)': 'humedad'
})

df

,provincia,temperatura,velocidad_viento,direccion_viento,racha_maxima,direccion_racha,precipitacion,presion,tendencia_presion,humedad
Estación,,,,,,,,,,
Abla,Almería,20.2,0,1-Calma,5,9-Noroeste,0.0,916.0,-0.4-0.4,38.0
Adra,Almería,21.3,17,8-Oeste,25,8-Oeste,0.0,1013.3,-0.2-0.2,86.0
Aguilar de la Frontera,Córdoba,17.7,17,8-Oeste,23,8-Oeste,0.0,,,81.0
Alajar,Huelva,16.6,13,2-Norte,24,3-Nordeste,0.0,950.3,1.0,62.0
Albox,Almería,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
Villanueva de Córdoba,Córdoba,20.3,13,6-Sur,28,7-Sudoeste,0.0,929.8,0.3,64.0
Villanueva del Arzobispo,Jaén,,,,,,,,,
Villarrasa,Huelva,19.1,12,2-Norte,22,2-Norte,0.0,,,58.0
